In [90]:
import requests
import base64
import json
import pandas
import os
import webbrowser
import urllib


def get_code(client_id, redirect_uri, scopes):
    params = {'response_type': 'code',
              'redirect_uri': redirect_uri,
              'client_id': client_id,
              'scope': scopes}
    url = 'https://accounts.spotify.com/authorize'
    requests.get(url=url, params=params)

    auth_url = url + '?' + urllib.urlencode(params)
    webbrowser.open(auth_url)

    try:
        response = raw_input("Enter the URL you were redirected to: ")
    except NameError:
        response = input("Enter the URL you were redirected to: ")

    code = response

    return code


def get_token(code, client_id, client_secret, redirect_uri):
    url = 'https://accounts.spotify.com/api/token'
    payload = {
        'grant_type': 'authorization_code',
        'code': code,
        'redirect_uri': redirect_uri
    }
    headers = {
        'Authorization': 'Basic ' + base64.b64encode(client_id + ':' + client_secret)
    }

    data = requests.post(url=url, data=payload, headers=headers)
    data = json.loads(data.text)

    access_token = data['access_token']

    return access_token


def get_playlists():
    url = 'https://api.spotify.com/v1/me/playlists'

    limit = 50
    offset = 0
    params = {'limit': limit,
              'offset': offset
              }
    data = requests.get(url=url, headers=header, params=params)
    playlist_data = json.loads(data.text)

    playlists = playlist_data['items']

    while playlist_data['next'] is not None:
        offset = offset + limit
        params = {'limit': limit,
                  'offset': offset}
        data = requests.get(url=url, headers=header, params=params)
        playlist_data = json.loads(data.text)
        playlists = playlists + playlist_data['items']

    return playlists


def get_playlist_tracks(user_id, playlist_id):
    url = 'https://api.spotify.com/v1/users/' + user_id + '/playlists/' + playlist_id + '/tracks'
    limit = 100
    offset = 0
    params = {'limit': limit,
              'offset': offset
              }
    data = requests.get(url=url, headers=header, params=params)
    track_data = json.loads(data.text)
    df = pandas.DataFrame(track_data['items'])
    track_info = pandas.DataFrame(df['track'].tolist())

    tracks = track_info[['id', 'name']]
    artists = track_info[['artists']].apply(lambda x: x[0], 1)['artists'].apply(pandas.Series)
    #artists = pandas.DataFrame(track_info['artists'].tolist())[['name']]
    tracks['artist'] = artists['name']
    return tracks


def get_track_info(track_ids):
    url = 'https://api.spotify.com/v1/audio-features'
    tracks = ",".join(track_ids)
    params = {
        'ids': tracks
    }
    data = requests.get(url, params=params, headers=header)
    feature_data = pandas.DataFrame(json.loads(data.text)['audio_features'])
    return feature_data



token = 'BQDtuvd-IWURxg1Tu53lPQ9eClm0woPpVlfTwWh1N3rPJ3XoXEqwbseEwEcKpJ9keZ2wg745h8qGm7hXsyYYT2DI_hfZtCwJ2buZd84mwOVbYarDmY3FuG77o-fot4AS07rh66Z9jc1-QLCPf_ou-Kw8jxJlAec'

global header
header = {'Authorization': 'Bearer ' + token}

playlists = get_playlists()




In [86]:
print playlists[18]
playlist = playlists[18]


{u'name': u'Kanye West \u2013 The Life of Pablo', u'collaborative': False, u'external_urls': {u'spotify': u'http://open.spotify.com/user/breakurjea/playlist/76xuHTl8iWuFTOAI5JVvSc'}, u'uri': u'spotify:user:breakurjea:playlist:76xuHTl8iWuFTOAI5JVvSc', u'public': True, u'owner': {u'display_name': u'Jason Jea', u'external_urls': {u'spotify': u'http://open.spotify.com/user/breakurjea'}, u'uri': u'spotify:user:breakurjea', u'href': u'https://api.spotify.com/v1/users/breakurjea', u'type': u'user', u'id': u'breakurjea'}, u'tracks': {u'total': 18, u'href': u'https://api.spotify.com/v1/users/breakurjea/playlists/76xuHTl8iWuFTOAI5JVvSc/tracks'}, u'href': u'https://api.spotify.com/v1/users/breakurjea/playlists/76xuHTl8iWuFTOAI5JVvSc', u'snapshot_id': u'o37pU6iVB5r9oJsyL1e8lYWwnmLzEzCaqteSiIzmk9LsvDNVGWxmahjXDG9K1Oc3', u'images': [], u'type': u'playlist', u'id': u'76xuHTl8iWuFTOAI5JVvSc'}


In [87]:
playlist_id = playlist['id']
user_id = playlist['owner']['id']
playlist_name = playlist['name']
print playlist_id
print user_id
print playlist_name

76xuHTl8iWuFTOAI5JVvSc
breakurjea
Kanye West – The Life of Pablo


In [106]:
playlist_id = '1oMEu0h82SQ4Y4Ci9kbNSB'
user_id = 'breakurjea'

url = 'https://api.spotify.com/v1/users/' + user_id + '/playlists/' + playlist_id + '/tracks'
limit = 100
offset = 0
params = {'limit': limit,
          'offset': offset
          }
data = requests.get(url=url, headers=header, params=params)
track_data = json.loads(data.text)
df = pandas.DataFrame(track_data['items'])
print df
print df.empty

Empty DataFrame
Columns: []
Index: []
True


In [93]:
track_info = pandas.DataFrame(df['track'].tolist())
print track_info

                                                album  \
0   {u'album_type': None, u'name': u'The Life of P...   
1   {u'album_type': None, u'name': u'The Life of P...   
2   {u'album_type': None, u'name': u'The Life of P...   
3   {u'album_type': None, u'name': u'The Life of P...   
4   {u'album_type': None, u'name': u'The Life of P...   
5   {u'album_type': None, u'name': u'The Life of P...   
6   {u'album_type': None, u'name': u'The Life of P...   
7   {u'album_type': None, u'name': u'The Life of P...   
8   {u'album_type': None, u'name': u'The Life of P...   
9   {u'album_type': None, u'name': u'The Life of P...   
10  {u'album_type': None, u'name': u'The Life of P...   
11  {u'album_type': None, u'name': u'The Life of P...   
12  {u'album_type': None, u'name': u'The Life of P...   
13  {u'album_type': None, u'name': u'The Life of P...   
14  {u'album_type': None, u'name': u'The Life of P...   
15  {u'album_type': None, u'name': u'The Life of P...   
16  {u'album_type': None, u'nam

In [101]:
tracks = track_info[['id', 'name']]
tracks = tracks[tracks['id'].notnull()]

Empty DataFrame
Columns: [id, name]
Index: []


In [97]:
print tracks
artists = track_info[['artists']].apply(lambda x: x[0][0]['name'], 1)
tracks.loc[:, 'artist'] = artists

url = 'https://api.spotify.com/v1/audio-features'
tracks = ",".join(tracks['id'])
params = {
    'ids': tracks
}
data = requests.get(url, params=params, headers=header)
feature_data = pandas.DataFrame(json.loads(data.text)['audio_features'])
print feature_data


      id                           name
0   None                Ultralight Beam
1   None  Father Stretch My Hands Pt. 1
2   None                          Pt. 2
3   None                         Famous
4   None                       Feedback
5   None                     Low Lights
6   None                     Highlights
7   None                    Freestyle 4
8   None                   I Love Kanye
9   None                          Waves
10  None                            FML
11  None                   Real Friends
12  None                         Wolves
13  None     Silver Surfer Intermission
14  None                       30 Hours
15  None          No More Parties In LA
16  None   Facts (Charlie Heat Version)
17  None                           Fade


TypeError: sequence item 0: expected string, NoneType found

In [78]:
track_features.loc[:, 'playlist_name'] = playlist_name
track_features.loc[:, 'playlist_id'] = playlist_id
track_features.loc[:, 'user_id'] = user_id
track_features = track_features.rename(columns={'id': 'track_id'})
track_features.to_csv('/Users/jasonjea/playlist-project/playlist_track_data.txt', sep='\t')